# Stochastic Unit Commitment Problem

In [1]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg

from sddip import config, storage, utils
from sddip import dualsolver, ucmodel, scenarios

## Data Processing

In [2]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, sep="\s+")
branch_df = pd.read_csv(branch_file_raw, sep="\s+")
gen_df = pd.read_csv(gen_file_raw, sep="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, sep="\s+")

scenario_df = pd.read_csv(scenario_data_file, sep="\s+")

### Bus Data

In [3]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


### Branch Data

In [4]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.04,0.2,0,990000,0,0,0,0,1,-360,360


### Generator Data

In [5]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [6]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [7]:
scenario_df

,t,n,p,Pd1,Pd2
0,1,1,1.0,0,170.926845
1,2,1,0.5,0,377.008048
2,2,2,0.5,0,316.678885
3,3,1,0.5,0,324.993856
4,3,2,0.5,0,379.699141


### Power Transfer Distribution Factor

In [8]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0., -1.]])

### Parameter Organization

In [9]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [10]:
# prob[t][n]
# Probability of realization n at stage t
prob

[[1.0], [0.5, 0.5], [0.5, 0.5]]

In [11]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
p_d

[[[0.0, 170.926844520483]],
 [[0.0, 377.0080482947398], [0.0, 316.6788854056216]],
 [[0.0, 324.9938564063665], [0.0, 379.6991413134704]]]

In [12]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
ex_pd

[array([  0.        , 170.92684452]),
 array([  0.        , 346.84346685]),
 array([  0.        , 352.34649886])]

## SDDiP

In [13]:
# Result keys
x_key = "x"
y_key = "y"
z_x_key = "zx"
z_y_key = "zy"
primal_solution_keys = [x_key, y_key, z_x_key, z_y_key]

dv_key = "dual_value"
dm_key = "dual_multiplier"
dual_solution_keys = [dv_key, dm_key]

ci_key = "intercepts"
cg_key = "gradients"
bm_key = "multipliers"
cut_coefficient_keys = [ci_key, cg_key, bm_key]


# Result storage
ps_storage = storage.ResultStorage(primal_solution_keys)
ds_storage = storage.ResultStorage(dual_solution_keys)
cc_storage = storage.ResultStorage(cut_coefficient_keys)


### Forward pass

In [14]:
penalty = 10000
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist() + [penalty]*2
theta_lb = 0
x_trial_point = [0]*n_gens
y_trial_point = [0]*n_gens


# Sampling
n_samples = 2
samples = scenarios.ScenarioSampler(n_stages, 2).generate_samples(n_samples)

v_opt_k = []
i = 0
for k in range(n_samples):
    v_opt_k.append(0)
    for t, n in zip(range(n_stages), samples[k]):

        # Create forward model
        uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

        uc_fw.add_objective(cost_coeffs)

        uc_fw.add_balance_constraints(sum(p_d[t][n]))

        uc_fw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

        uc_fw.add_generator_constraints(pg_min, pg_max)

        uc_fw.add_startup_shutdown_constraints()

        uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

        uc_fw.add_copy_constraints(x_trial_point, y_trial_point)

        #TODO Lower bound
        uc_fw.add_cut_lower_bound(theta_lb)
        
        if i>0:
            cut_coefficients = cc_storage.get_stage_result(t)
            uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])
        
        # Solve problem
        uc_fw.disable_output()
        uc_fw.model.optimize()
        model = uc_fw.model
        uc_fw.model.printAttr("X")

        # Store xtik, ytik, ztik, vtik
        x_kt = [x_g.x for x_g in uc_fw.x]
        y_kt = [y_g.x for y_g in uc_fw.y]
        z_x_kt = [z_g.x for z_g in uc_fw.z_x]
        z_y_kt = [z_g.x for z_g in uc_fw.z_y]
        s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
        s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]
        
        # Value of stage t objective function
        print(f"Objective value t={t}, n={n}: {uc_fw.model.getObjective().getValue()}")
        v_opt_kt = uc_fw.model.getObjective().getValue() - uc_fw.theta.x

        v_opt_k[-1] += v_opt_kt

        # New trial point
        x_trial_point = x_kt
        y_trial_point = y_kt

        ps_dict = ps_storage.create_empty_result_dict()
        ps_dict[x_key] = x_kt
        ps_dict[y_key] = y_kt
        ps_dict[z_x_key] = z_x_kt
        ps_dict[z_y_key] = z_y_kt
        
        ps_storage.add_result(i, k, t, ps_dict)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-01-25
Objective value t=0, n=0: 341.853689040966
Objective value t=1, n=1: 633.3577708112432
Objective value t=2, n=0: 649.987712812733
Objective value t=0, n=0: 341.853689040966
Objective value t=1, n=0: 754.0160965894796
Objective value t=2, n=1: 759.3982826269408


#### Statistical Upper Bound

In [15]:
v_opt_k = np.array(v_opt_k)

v_mean = np.mean(v_opt_k)
v_std = np.std(v_opt_k)
alpha = 0.05

# Lower limit of the confidence interval
v_upper = v_mean + stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)
v_upper

1580.8069393284266

In [16]:
# Upper limit of the confidence interval
v_mean - stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)

1899.660301593902

In [17]:
primal_solution_df = ps_storage.to_dataframe()
primal_solution_df

x                    y     zx                   zy
i k t                                                        
0 0 0  [1.0]   [170.926844520483]  [0.0]                [0.0]
    1  [1.0]  [316.6788854056216]  [1.0]   [170.926844520483]
    2  [1.0]  [324.9938564063665]  [1.0]  [316.6788854056216]
  1 0  [1.0]   [170.926844520483]  [1.0]  [324.9938564063665]
    1  [1.0]  [377.0080482947398]  [1.0]   [170.926844520483]
    2  [1.0]  [379.6991413134704]  [1.0]  [377.0080482947398]

### Backward pass

In [18]:
########################################################################################################################
# Backward pass
########################################################################################################################


binarizer = utils.Binarizer()

sg_method = dualsolver.SubgradientMethod(max_iterations=10)

cut_lower_bounds = [theta_lb]*n_stages
v_opt_k =[]


i = 0

for t in reversed(range(1,n_stages)):
    for k in range(n_samples):
        n_realizations = n_nodes_per_stage[t]
        ds_dict = ds_storage.create_empty_result_dict()
        cc_dict = cc_storage.create_empty_result_dict()
        
        for n in range(n_realizations):

            # TODO Binarization
            bin_vars = []
            bin_multipliers = []
            if t>0:
                float_vars = ps_storage.get_result(i,k,t-1)[y_key]
                x_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_key]
            else:
                #TODO Approximation needed?
                # Might lead to active penalty
                float_vars = np.zeros(n_gens)
                x_binary_trial_point = np.zeros(n_gens)
            
            for j in range(len(float_vars)):
                new_vars, new_multipliers = binarizer.binary_expansion(float_vars[j], upper_bound=pg_max, precision=0.1)
                bin_vars += new_vars
                bin_multipliers.append(new_multipliers) 


            y_binary_trial_point = bin_vars
            y_binary_trial_multipliers = linalg.block_diag(*bin_multipliers)


            uc_bw = ucmodel.BackwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

            uc_bw.add_objective(cost_coeffs)

            uc_bw.add_balance_constraints(sum(p_d[t][n]))

            uc_bw.add_generator_constraints(pg_min, pg_max)

            uc_bw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

            uc_bw.add_startup_shutdown_constraints()

            uc_bw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

            uc_bw.add_relaxation(x_binary_trial_point, y_binary_trial_point)

            uc_bw.add_copy_constraints(y_binary_trial_multipliers)

            uc_bw.add_cut_lower_bound(cut_lower_bounds[t])
            
            if t < n_stages-1:
                cut_coefficients = cc_storage.get_stage_result(t)
                uc_bw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key], sos=True)

            objective_terms = uc_bw.objective_terms
            relaxed_terms = uc_bw.relaxed_terms
            
            # Solve problem with subgradient method
            uc_bw.disable_output()
            uc_bw.model.optimize()
            uc_bw.model.display()
            
            sg_method.output_flag = True            
            model, sg_results = sg_method.solve(uc_bw.model, objective_terms, relaxed_terms, 2000)
            print(f"t={t}, n={n}")
            print(f"Best lower bound:{sg_results.obj_value}")
            print(f"Best multipliers: {sg_results.multipliers}")


            # if t== 1 and n == 0:
            #     model.setParam("OutputFlag", 1)
            #     model.printAttr("X")
            #     model.display()

            
            # Dual value and multiplier for each realization
            ds_dict[dv_key].append(sg_results.obj_value)
            ds_dict[dm_key].append(sg_results.multipliers)
            
        
        ds_storage.add_result(i, k, t, ds_dict)

        # Calculate and store cut coefficients
        probabilities = prob[t]        
        intercept = np.array(probabilities).dot(np.array(ds_dict[dv_key]))
        gradient = np.array(probabilities).dot(np.array(ds_dict[dm_key]))

        cc_dict[ci_key] = intercept.tolist()
        cc_dict[cg_key] = gradient.tolist()
        cc_dict[bm_key] = y_binary_trial_multipliers

        if t > 0 : cc_storage.add_result(i, k, t-1, cc_dict)
                

Subgradient Method started
Iteration: 0 | Optimal value: 649.987712812733 | Gradient magnitude: 3.605551275463989 | Step size: 0.2773500981126146
Iteration: 1 | Optimal value: 649.9877128127331 | Gradient magnitude: 0.0
Subgradient Method finished (Tolerance)
t=2, n=0
Best lower bound:649.9877128127331
Best multipliers: [0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501
 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.       ]
Subgradient Method started
Iteration: 0 | Optimal value: 759.3982826269408 | Gradient magnitude: 3.605551275463989 | Step size: 0.2773500981126146
Iteration: 1 | Optimal value: 759.398282626941 | Gradient magnitude: 0.0
Subgradient Method finished (Tolerance)
t=2, n=1
Best lower bound:759.398282626941
Best multipliers: [0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501
 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.2773501 0.       ]
Subgradient Method started
Iteration: 0 | Optimal value: 649.9877128127

In [19]:
########################################################################################################################
# Backward Primal solutions
########################################################################################################################


binarizer = utils.Binarizer()

sg_method = dualsolver.SubgradientMethod(max_iterations=10)

cut_lower_bounds = [theta_lb]*n_stages
v_opt_k =[]


i = 0
for t in reversed(range(1,n_stages)):
    for k in range(n_samples):
        n_realizations = n_nodes_per_stage[t]
        ds_dict = ds_storage.create_empty_result_dict()
        cc_dict = cc_storage.create_empty_result_dict()
        
        for n in range(n_realizations):

            # TODO Binarization
            bin_vars = []
            bin_multipliers = []
            if t>0:
                float_vars = ps_storage.get_result(i,k,t-1)[y_key]
                x_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_key]
            else:
                #TODO Approximation needed?
                # Might lead to active penalty
                float_vars = np.zeros(n_gens)
                x_binary_trial_point = np.zeros(n_gens)
            
            for j in range(len(float_vars)):
                new_vars, new_multipliers = binarizer.binary_expansion(float_vars[j], upper_bound=pg_max, precision=0.1)
                bin_vars += new_vars
                bin_multipliers.append(new_multipliers) 


            y_binary_trial_point = bin_vars
            y_binary_trial_multipliers = linalg.block_diag(*bin_multipliers)


            uc_bw = ucmodel.BackwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

            uc_bw.add_objective(cost_coeffs)

            uc_bw.add_balance_constraints(sum(p_d[t][n]))

            uc_bw.add_generator_constraints(pg_min, pg_max)

            uc_bw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

            uc_bw.add_startup_shutdown_constraints()

            uc_bw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

            uc_bw.add_relaxation(x_binary_trial_point, y_binary_trial_point)

            uc_bw.add_copy_constraints(y_binary_trial_multipliers)

            uc_bw.add_cut_lower_bound(cut_lower_bounds[t])
            
            if t < n_stages-1:
                cut_coefficients = cc_storage.get_stage_result(t)
                uc_bw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key], sos=True)

            objective_terms = uc_bw.objective_terms
            relaxed_terms = uc_bw.relaxed_terms
            
            for rt in relaxed_terms:
                uc_bw.model.addConstr(rt==0)
            
            uc_bw.model.optimize()
            obj_val = uc_bw.model.getObjective().getValue()
            print(f"Primal optimal value t={t}, n={n}: {obj_val}")

Primal optimal value t=2, n=0: 649.987712812733
Primal optimal value t=2, n=1: 759.3982826269408
Primal optimal value t=2, n=0: 649.987712812733
Primal optimal value t=2, n=1: 759.3982826269408
Primal optimal value t=1, n=0: 1462.2706429966042
Primal optimal value t=1, n=1: 1341.5306169489286
Primal optimal value t=1, n=0: 1462.2706429966042
Primal optimal value t=1, n=1: 1341.5306169489286


In [20]:
# One dual value and one set of dual multipliers for each realization
ds_storage.to_dataframe()

dual_value  \
i k t                                             
0 0 2     [649.9877128127331, 759.398282626941]   
  1 2     [649.9877128127331, 759.398282626941]   
  0 1  [1462.2706434065103, 1341.5306169483601]   
  1 1  [1462.2706434065103, 1341.5306169483601]   

                                         dual_multiplier  
i k t                                                     
0 0 2  [[0.2773500981126146, 0.2773500981126146, 0.27...  
  1 2  [[0.2773500981126146, 0.2773500981126146, 0.27...  
  0 1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
  1 1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...

In [21]:
cc_storage.to_dataframe()

intercepts                                          gradients  \
i k t                                                                   
0 0 1   704.692998  [0.2773500981126146, 0.2773500981126146, 0.277...   
  1 1   704.692998  [0.2773500981126146, 0.2773500981126146, 0.277...   
  0 0  1401.900630  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
  1 0  1401.900630  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                             multipliers  
i k t                                                     
0 0 1  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....  
  1 1  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....  
  0 0  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....  
  1 0  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....

### Lower Bound Update

In [22]:
i = 1
t = 0
n = 0

x_trial_point = [0]*n_gens
y_trial_point = [0]*n_gens

# Create forward model
uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

uc_fw.add_objective(cost_coeffs)

uc_fw.add_balance_constraints(sum(p_d[t][n]))

uc_fw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

uc_fw.add_generator_constraints(pg_min, pg_max)

uc_fw.add_startup_shutdown_constraints()

uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

uc_fw.add_copy_constraints(x_trial_point, y_trial_point)

#TODO Lower bound
uc_fw.add_cut_lower_bound(theta_lb)

if i>0:
    cut_coefficients = cc_storage.get_stage_result(t)
    uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key], sos=True)

# Solve problem
#uc_fw.disable_output()
uc_fw.model.optimize()
uc_fw.model.printAttr("X")
    


# Store xtik, ytik, ztik, vtik
x_kt = [x_g.x for x_g in uc_fw.x]
y_kt = [y_g.x for y_g in uc_fw.y]
z_x_kt = [z_g.x for z_g in uc_fw.z_x]
z_y_kt = [z_g.x for z_g in uc_fw.z_y]
s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]

# Value of stage t objective function
v_lower = uc_fw.model.getObjective().getValue()

v_lower
#TODO : Problem cut ist keine Funktion in y, sondern scalar

1743.7543192184012

In [23]:
uc_fw.enable_output()
uc_fw.model.printAttr("X")

Set parameter OutputFlag to value 1

    Variable            X 
-------------------------
         x_1            1 
         y_1      170.927 
      s_up_1            1 
       theta       1401.9 
 lambda_0[0]            1 
 lambda_0[8]            1 
 lambda_0[9]            1 
lambda_0[10]            1 
lambda_0[11]     0.794207 
      w_0[0]            1 
      w_0[8]            1 
      w_0[9]            1 
     w_0[10]            1 
     w_0[11]            1 
      u_0[1]            1 
      u_0[2]            1 
      u_0[3]            1 
      u_0[4]            1 
      u_0[5]            1 
      u_0[6]            1 
      u_0[7]            1 
      u_0[8]            1 
      u_0[9]            1 
     u_0[10]            1 
     u_0[11]            1 
     u_0[12]            1 
     u_0[13]            1 
 lambda_1[0]            1 
 lambda_1[8]            1 
 lambda_1[9]            1 
lambda_1[10]            1 
lambda_1[11]     0.794207 
      w_1[0]            1 
      w_1[8]       

In [24]:
uc_fw.model.display()

Minimize
  <gurobi.LinExpr: 2.0 y_1 + theta + 10000.0 ys_p + 10000.0 ys_n>
Subject To
  balance: <gurobi.LinExpr: y_1 + ys_p + -1.0 ys_n> = 170.927
  power-flow(1)[0]: <gurobi.LinExpr: 0.0> <= 989829
  power-flow(2)[0]: <gurobi.LinExpr: 0.0> <= 990171
  min-generation[0]: <gurobi.LinExpr: y_1> >= 0
  max-generation[0]: <gurobi.LinExpr: -600.0 x_1 + y_1> <= 0
  up-down(1)[0]: <gurobi.LinExpr: x_1 + -1.0 s_up_1 + -1.0 z_x_1> <= 0
  up-down(2)[0]: <gurobi.LinExpr: x_1 + -1.0 s_up_1 + s_down_1 + -1.0 z_x_1> = 0
  rate-up[0]: <gurobi.LinExpr: y_1 + -1.0 z_y_1> <= 1000
  rate-down[0]: <gurobi.LinExpr: -1.0 y_1 + z_y_1> <= 1000
  copy-x[0]: <gurobi.LinExpr: z_x_1> = 0
  copy-y[0]: <gurobi.LinExpr: z_y_1> = 0
  cut-lb: <gurobi.LinExpr: theta> >= 0
  cut_0: <gurobi.LinExpr: theta> >= 1401.9
  KKT(1)_0[0]: <gurobi.LinExpr: -1.0 ny_0[0] + my_0[0] + eta_0[0]> = 0
  KKT(1)_0[1]: <gurobi.LinExpr: -1.0 ny_0[1] + my_0[1] + 0.1 eta_0[1]> = 0
  KKT(1)_0[2]: <gurobi.LinExpr: -1.0 ny_0[2] + my_0[2] + 0.2 

In [25]:
# model.setParam("DualReductions", 0)
# model.reset()
# model.optimize()

In [26]:
# model.computeIIS()
# if model.IISMinimal:
#     print("IIS is minimal.")
# else:
#     print("IIS is not minimal.")
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s'%model.constrName)

In [27]:
# model.write("iis"+".ilp")

In [28]:
#df = ps_storage.to_dataframe()
#df

In [29]:
df = ps_storage.to_dataframe()
df

x                    y     zx                   zy
i k t                                                        
0 0 0  [1.0]   [170.926844520483]  [0.0]                [0.0]
    1  [1.0]  [316.6788854056216]  [1.0]   [170.926844520483]
    2  [1.0]  [324.9938564063665]  [1.0]  [316.6788854056216]
  1 0  [1.0]   [170.926844520483]  [1.0]  [324.9938564063665]
    1  [1.0]  [377.0080482947398]  [1.0]   [170.926844520483]
    2  [1.0]  [379.6991413134704]  [1.0]  [377.0080482947398]